In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import numpy as np
import pandas as pd

In [4]:
from ecephys.data import paths
from ecephys.scoring import load_datetime_hypnogram, filter_states
from ecephys.utils import load_df_h5

from ecephys.signal.sharp_wave_ripples import get_events_in_interval
import ecephys.plot as eplt

In [5]:
subject = "Doppio"
condition = "recovery-sleep-6h"

In [13]:
hypnogram_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="hypnogram.tsv")
hypnograms = [load_datetime_hypnogram(path) for path in hypnogram_paths]

In [14]:
hypnogram = pd.concat(hypnograms).reset_index(drop=True)
condition_start_dt = hypnogram['start_time'].iloc[0]

In [15]:
spw_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="spws.h5")
spws = [load_df_h5(path) for path in spw_paths]

In [16]:
for _spws, _hypnogram in zip(spws, hypnograms):
    file_start_dt = _hypnogram['start_time'].min()
    _spws['start_time'] = file_start_dt + pd.to_timedelta(_spws['start_time'], 's')
    _spws['end_time'] = file_start_dt + pd.to_timedelta(_spws['end_time'], 's')
    _spws['midpoint'] = file_start_dt + pd.to_timedelta(_spws['midpoint'], 's')
    #spws['duration'] = pd.to_timedelta(spws['duration'], 's')
    _spws['time_from_condition_start'] = _spws['start_time'] - condition_start_dt
    
combined_spws = pd.concat(spws).reset_index(drop=True)
combined_spws.index += 1
combined_spws.index = combined_spws.index.rename('spw_number')
spws = combined_spws

In [17]:
# density = spws.set_index('start_time').resample('10s').count().mean(axis=1).to_frame(name='frequency').reset_index()

In [18]:
fig, (density_ax, duration_ax, amplitude_ax, integral_ax) = plt.subplots(4, 1, figsize=(20,8))

xmargin=0
binwidth = 10
nbins = np.int((spws.start_time.max() - spws.start_time.min()).total_seconds() / binwidth)
sns.histplot(data=spws, x='start_time', bins=nbins, color='black', ax=density_ax)
density_ax.set(xlabel=None, ylabel='Frequency (Hz)', xmargin=xmargin, xticks=[])

sns.scatterplot(data=spws, x="start_time", y="duration", s=2, ax=duration_ax)
duration_ax.set(xlabel=None, ylabel="Duration (s)", xmargin=xmargin, xticks=[])

sns.scatterplot(data=spws, x='start_time', y='sink_amplitude', s=2, ax=amplitude_ax)
amplitude_ax.set(xlabel=None, ylabel="Sink amplitude (mA/mm)", xmargin=xmargin, xticks=[])

sns.scatterplot(data=spws, x='start_time', y='sink_integral', s=2, ax=integral_ax)
integral_ax.set(xlabel="Event time (month-date hour)", ylabel="Sink integral (mA * ms)", xmargin=xmargin)

for ax in (density_ax, duration_ax, amplitude_ax, integral_ax):
    eplt.plot_hypnogram_overlay(hypnogram, ax=ax)

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
def add_spw_properties_to_hypnogram(spws, hypnogram):
    for i, bout in hypnogram.iterrows():
        bout_events = get_events_in_interval(spws, bout.start_time, bout.end_time)
        hypnogram.at[i, 'spw_density'] = len(bout_events) / bout.duration.total_seconds()
        hypnogram.at[i, 'mean_duration'] = bout_events.duration.mean()
        hypnogram.at[i, 'mean_sink_amplitude'] = bout_events.sink_amplitude.mean()
        hypnogram.at[i, 'mean_sink_integral'] = bout_events.sink_integral.mean()
    
    hypnogram.spw_density.fillna(value=0, inplace=True)
    hypnogram.mean_duration.fillna(value=0, inplace=True)
    hypnogram.mean_sink_amplitude.fillna(value=0, inplace=True)
    hypnogram.mean_sink_integral.fillna(value=0, inplace=True)

In [43]:
add_spw_properties_to_hypnogram(spws, hypnogram)

In [44]:
qwk = filter_states(hypnogram, ['qWk']).reset_index(drop=True)

In [45]:
def plot_spw_summary_by_bout_rank(hypnogram, figsize=(20, 8)):
    fig, (density_ax, duration_ax, amplitude_ax, integral_ax) = plt.subplots(4, 1, figsize=figsize)
    
    hypnogram.plot.scatter(x='index', y='spw_density', ax=density_ax, title='SPW density (Hz)', s=1)
    hypnogram.plot.scatter(x='index', y='mean_duration', ax=duration_ax, title='Event duration (s)', s=1)
    hypnogram.plot.scatter(x='index', y='mean_sink_amplitude', ax=amplitude_ax, title='Sink amplitude (mA/mm)', s=1)
    hypnogram.plot.scatter(x='index', y='mean_sink_integral', ax=integral_ax, title='Sink integral (mA * ms)', s=1)
    for ax in (density_ax, duration_ax, amplitude_ax, integral_ax):
        ax.set_xlabel('Bout rank')
    plt.tight_layout()

In [46]:
plot_spw_summary_by_bout_rank(qwk.reset_index())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
awk = filter_states(hypnogram, ['aWk']).reset_index(drop=True)

In [48]:
plot_spw_summary_by_bout_rank(awk.reset_index())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …